In [1]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union, Dict, List
import itertools 
import glob

import warnings
import time

import numpy as np
import pandas as pd

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files,
                                        read_tiff_image)



from infer_subc.constants import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats_helpers import *
from infer_subc.utils.stats import _assert_uint16_labels
from infer_subc.core.img import label_uint16


import time
%load_ext autoreload
%autoreload 2

__________________________
## Collecting Summary Stats across multiple experiments - Part 1.6

---------
## **Batch Summary Stats**

### summary of steps

🛠️ BUILD FUNCTION PROTOTYPE

- **`0`** - Establish csv paths *(preliminary step)*

- **`1`** - Read in and categorize csv files

    - read in and categorize csv files for all listed paths
    - Combine comprehensive metrics tables to be summarized and restructured

- **`2`** - Restructure comprehensive organelle two-way interaction metrics table

    - breakdown the interaction table column names
    - group observations by the **first** organelle involved in interactions
    - unstack the grouped table to create a column for every unique organelle interaction type
    - correct column names for unstacked tables to accurately describe each the **first** organelles involved in interaction sites
    - repeat last three substeps for the **second** organelle involved in the interactions
    - combine and merge the data from **both** unstacked tables to include interaction metrics from all organelle objects

- **`3`** - Apply aggregate statistics for summarization

    - determine aggregate statistics to be applied per organelle object
    - summarize metrics between the organelle morphology and interaction tables
    - summarize metrics in the region morphology table
    - summarize additional metrics in the organelle morphology table

- **`4`** - Restructure distribution metrics tables

    - for XY-distribution collect summary statistics for voxel bins and wedges
    - for Z-distribution collect summary statistics for voxel bins and wedges
    - calculate the coefficient of variation for the **mean**, **median**, **standard deviation** for the XY-distribution bin values
    - repeat the first two substeps for the nucleus distribution metrics
    - combine nucleus and organelle distribution tables


- **`5`** - Add normalized metrics

    - calculate fraction of cell area taken up by the organelles
    - calculate fraction of organelle objects involved in specific interorganelle contacts

- **`6`** - Unstack and finalize summary stats tables

    - unstack and reorder organelle morphology summary table columns
    - fill "NaN" values with 0 when necessary to final organelle morphology summary table
    - unstack and reorder organelle interactions summary table columns
    - fill "NaN" values with 0 when necessary to final organelle interactions summary table
    - unstack and reorder distribution measurements summary table columns to create finalized table
    - unstack and reorder region morphology summary table columns
    - add normalization to finalize region morphology summary table
    - combine all four tables to create a complete summary table

- **`7`** - Export summary stats tables as .csv files

⚙️ EXECUTE FUNCTION PROTOTYPE

- Define prototype `_batch_summary_stats` function

- Run prototype `_batch_summary_stats` function

# ***BUILD FUNCTION PROTOTYPE***

## **`0` - Establish csv paths *(preliminary step)***

In [2]:
# List of file paths to be included in the summary
csv_path_list=["F:/Experiments (C2-80 - C2-108)/C2-107/20230919_C2-107_3D-analysis/20240217_C2-107_quantification_10per",
                "F:/Experiments (C2-80 - C2-108)/C2-108/20230920_C2-108_3D-analysis/20240216_C2-108_qantification_10per",
                "D:/Experiments (C2-117 - current)/C2-123/20230922_C2-123_3D-analysis/20240118_C2-123_quantification",
                "D:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/20240221_C2-121_quantification_10per",
                "D:/Experiments (C2-117 - current)/C2-117/20230922_C2-117_3D-analysis/20240118_C2-117_quantification"]

## **`1` - Read in and categorize csv files**

- read in and categorize csv files for all listed paths

In [ ]:
org_tabs = []
contact_tabs = []
dist_tabs = []
region_tabs = []

for loc in csv_path_list:
    ds_count = ds_count + 1
    loc=Path(loc)
    files_store = sorted(loc.glob("*.csv"))
    for file in files_store:
        fl_count = fl_count + 1
        stem = file.stem

        org = "organelles"
        contacts = "contacts"
        dist = "distributions"
        regions = "_regions"

        if org in stem:
            test_orgs = pd.read_csv(file, index_col=0)
            test_orgs.insert(0, "dataset", stem[:-11])
            org_tabs.append(test_orgs)
        if contacts in stem:
            test_contact = pd.read_csv(file, index_col=0)
            test_contact.insert(0, "dataset", stem[:-9])
            contact_tabs.append(test_contact)
        if dist in stem:
            test_dist = pd.read_csv(file, index_col=0)
            test_dist.insert(0, "dataset", stem[:-14])
            dist_tabs.append(test_dist)
        if regions in stem:
            test_regions = pd.read_csv(file, index_col=0)
            test_regions.insert(0, "dataset", stem[:-8])
            region_tabs.append(test_regions)


- Combine comprehensive metrics tables to be summarized and restructured

In [ ]:
org_df = pd.concat(org_tabs,axis=0, join='outer')
contacts_df = pd.concat(contact_tabs,axis=0, join='outer')
dist_df = pd.concat(dist_tabs,axis=0, join='outer')
regions_df = pd.concat(region_tabs,axis=0, join='outer')

In [3]:
org_df.head()

,dataset,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,...,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,LD,34,"(0.4106, 0.0799, 0.0799)",5.748316,79.220209,66.087097,14,990,...,30.710824,0.380601,0.916667,1,inf,0.388472,3209.564941,4803.942871,3969.853027,513.974487
1,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,LD,41,"(0.4106, 0.0799, 0.0799)",6.158910,80.546603,74.230790,15,1007,...,44.560389,0.271660,1.000000,1,inf,0.178767,3409.148438,3808.100098,3615.759521,167.172455
2,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,LD,46,"(0.4106, 0.0799, 0.0799)",6.569504,65.396646,64.373324,16,817,...,32.012333,0.368700,0.833333,1,inf,0.350310,3437.308105,5740.641113,4511.916992,669.799744
3,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,ER,1,"(0.4106, 0.0799, 0.0799)",7.077805,75.250102,70.922115,8,0,...,4.717567,14.705835,0.007423,-56,0.019041,83.485697,0.882749,9488.813477,1570.648071,1015.394409
4,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,golgi,1,"(0.4106, 0.0799, 0.0799)",6.518533,71.400411,65.367354,12,855,...,7.743745,2.856022,0.101596,-8,0.327393,8.271302,1037.344482,7995.974609,2276.678223,1043.566162


In [4]:
contacts_df.head()

,dataset,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,...,bbox-4,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length
0,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,LDXER,34_1,"(0.4106, 0.0799, 0.0799)",5.748316,79.174179,66.049547,14,990,...,992,829,0.701645,0.015746,44.560389,0.310973,0.750000,1,inf,0.390010
1,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,LDXER,41_1,"(0.4106, 0.0799, 0.0799)",6.158910,80.546603,74.230790,15,1007,...,1009,930,0.467763,0.010497,44.560389,0.271660,1.000000,1,inf,0.178767
2,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,LDXER,46_1,"(0.4106, 0.0799, 0.0799)",6.569504,65.396646,64.373324,16,817,...,820,808,0.840106,0.026243,32.012333,0.368700,0.833333,1,inf,0.350310
3,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,LDXlyso,34_4,"(0.4106, 0.0799, 0.0799)",5.748316,79.220209,66.087097,14,990,...,993,829,0.886546,0.028868,30.710824,0.380601,0.916667,1,inf,0.388472
4,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,ERXgolgi,1_1,"(0.4106, 0.0799, 0.0799)",5.794840,69.793156,67.015573,12,855,...,896,852,31.132222,1.459122,21.336265,1.407228,0.083710,-3,0.18873,4.252173


In [5]:
dist_df.head()

,dataset,image_name,object,scale,XY_n_bins,XY_bins,XY_mask_vox_cnt_perbin,XY_obj_vox_cnt_perbin,XY_center_vox_cnt_perbin,XY_n_pix_perbin,...,XY_area_wedges_perbin,Z_n_slices,Z_slices,Z_mask_vox_cnt,Z_obj_vox_cnt,Z_center_vox_cnt,Z_height,Z_mask_volume,Z_obj_volume,Z_center_volume
0,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,LD,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[427967, 140309, 156031, 169187, 919030]","[0, 0, 4, 21, 0]","[146905, 0, 0, 0, 0]","[16331.0, 6640.0, 9089.0, 12684.0, 171328.0]",...,"[[14.508756776430001, 12.380379681033002, 11.1...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 1254, 28021, 58752, 7916...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",16.013166,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.29089842...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,ER,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[427967, 140309, 156031, 169187, 919030]","[111076, 75286, 90475, 94836, 262855]","[146905, 0, 0, 0, 0]","[16331.0, 6640.0, 9089.0, 12684.0, 171328.0]",...,"[[14.508756776430001, 12.380379681033002, 11.1...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 1254, 28021, 58752, 7916...","[0, 0, 0, 0, 0, 0, 0, 0, 128, 2355, 15555, 385...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",16.013166,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.29089842...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.335...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,golgi,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[427967, 140309, 156031, 169187, 919030]","[3103, 8700, 4532, 987, 0]","[146905, 0, 0, 0, 0]","[16331.0, 6640.0, 9089.0, 12684.0, 171328.0]",...,"[[14.508756776430001, 12.380379681033002, 11.1...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 1254, 28021, 58752, 7916...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 400, 1437...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",16.013166,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.29089842...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,lyso,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[427967, 140309, 156031, 169187, 919030]","[16963, 7188, 7259, 8103, 9731]","[146905, 0, 0, 0, 0]","[16331.0, 6640.0, 9089.0, 12684.0, 171328.0]",...,"[[14.508756776430001, 12.380379681033002, 11.1...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 1254, 28021, 58752, 7916...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 136, 1517, 3608, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",16.013166,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.29089842...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,mito,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[427967, 140309, 156031, 169187, 919030]","[19049, 11861, 15057, 16599, 58895]","[146905, 0, 0, 0, 0]","[16331.0, 6640.0, 9089.0, 12684.0, 171328.0]",...,"[[14.508756776430001, 12.380379681033002, 11.1...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 1254, 28021, 58752, 7916...","[0, 0, 0, 0, 0, 0, 0, 0, 6, 853, 4861, 9597, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",16.013166,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.29089842...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 8_cell 5_un...,golgiXmito,"(0.4106, 0.0799, 0.0799)",

In [6]:
regions_df.head()

,dataset,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,...,mean_intensity-golgi_ch,mean_intensity-lyso_ch,mean_intensity-mito_ch,mean_intensity-perox_ch,standard_deviation_intensity-LD_ch,standard_deviation_intensity-ER_ch,standard_deviation_intensity-golgi_ch,standard_deviation_intensity-lyso_ch,standard_deviation_intensity-mito_ch,standard_deviation_intensity-perox_ch
0,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,nuc,1,"(0.4106, 0.0799, 0.0799)",9.656524,73.476817,72.015354,15,848,...,0.003980,0.033965,10.327600,16.790661,0.011877,0.435200,0.003469,0.192766,10.107121,15.170704
1,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 1_2...,cell,1,"(0.4106, 0.0799, 0.0799)",7.245385,74.949189,71.010845,7,0,...,32.623714,57.594608,133.466919,376.963348,72.220337,934.518188,244.761063,292.651947,293.679840,786.370483
2,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 2_2...,nuc,1,"(0.4106, 0.0799, 0.0799)",10.791034,67.889576,85.661055,14,782,...,0.110757,0.092745,51.535671,113.979195,0.161533,9.465900,0.246002,0.151073,36.946209,64.928368
3,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 2_2...,cell,1,"(0.4106, 0.0799, 0.0799)",6.552921,66.637646,74.770419,1,0,...,234.530655,39.489193,450.387329,1322.078247,138.427841,5498.334961,1012.422791,167.863388,830.095337,2822.268066
4,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 10_cell 3_2...,nuc,1,"(0.4106, 0.0799, 0.0799)",10.186749,60.882566,68.357664,15,697,...,0.027503,0.056172,47.277412,195.726944,0.064345,3.224035,0.063066,0.091853,32.378677,78.064690


## **`2` - Restructure comprehensive two-way organelle interaction metrics table**

> ###### **📝 Please note that in the following steps, a specific procedure will be repeated to ensure that all unique organelle objects are described by their interaction metrics, regardless of whether they are the first organelle (A) or the second organelle (B) involved in the two-way contact.**

- breakdown the interaction table column names

In [7]:
contact_cnt = contacts_df[["dataset", "image_name", "object", "label", "volume"]]
contact_cnt[["orgA", "orgB"]] = contact_cnt["object"].str.split('X', expand=True)
contact_cnt[["A_ID", "B_ID"]] = contact_cnt["label"].str.split('_', expand=True)
contact_cnt["A"] = contact_cnt["orgA"] +"_" + contact_cnt["A_ID"].astype(str)
contact_cnt["B"] = contact_cnt["orgB"] +"_" + contact_cnt["B_ID"].astype(str)

C:\Users\Shannon\AppData\Local\Temp\ipykernel_30264\3488508820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contact_cnt[["orgA", "orgB"]] = contact_cnt["object"].str.split('X', expand=True)
C:\Users\Shannon\AppData\Local\Temp\ipykernel_30264\3488508820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contact_cnt[["orgA", "orgB"]] = contact_cnt["object"].str.split('X', expand=True)
C:\Users\Shannon\AppData\Local\Temp\ipykernel_30264\3488508820.py:3: SettingWithCopyWarning: 
A value is trying to be 

- group observations by the **first** organelle involved in interactions

In [ ]:
contact_cnt_percell = contact_cnt[["dataset", "image_name", "orgA", "A_ID", "object", "volume"]].groupby(["dataset", "image_name", "orgA", "A_ID", "object"]).agg(["count", "sum"])
contact_cnt_percell.columns = ["_".join(col_name).rstrip('_') for col_name in contact_cnt_percell.columns.to_flat_index()]

- unstack the grouped table to create a column for every unique organelle interaction type

In [ ]:
unstacked = contact_cnt_percell.unstack(level='object')
unstacked.columns = ["_".join(col_name).rstrip('_') for col_name in unstacked.columns.to_flat_index()]
unstacked = unstacked.reset_index()

- correct column names for unstacked tables to accurately describe each the **first** organelles involved in interaction sites

In [ ]:
# Fixes the count and volume metrics
for col in unstacked.columns:
    if col.startswith("volume_count_"):
        newname = col.split("_")[-1] + "_count"
        unstacked.rename(columns={col:newname}, inplace=True)
    if col.startswith("volume_sum_"):
        newname = col.split("_")[-1] + "_volume"
        unstacked.rename(columns={col:newname}, inplace=True)

# first organelle is simply referred to as object
# label of first organelle is simply reffered to as label
unstacked.rename(columns={"orgA":"object", "A_ID":"label"}, inplace=True)
unstacked.set_index(['dataset', 'image_name', 'object', 'label'])


- repeat last three substeps for the **second** organelle involved in the interactions

In [ ]:
contact_percellB = contact_cnt[["dataset", "image_name", "orgB", "B_ID", "object", "volume"]].groupby(["dataset", "image_name", "orgB", "B_ID", "object"]).agg(["count", "sum"])
contact_percellB.columns = ["_".join(col_name).rstrip('_') for col_name in contact_percellB.columns.to_flat_index()]
unstackedB = contact_percellB.unstack(level='object')
unstackedB.columns = ["_".join(col_name).rstrip('_') for col_name in unstackedB.columns.to_flat_index()]
unstackedB = unstackedB.reset_index()
for col in unstackedB.columns:
    if col.startswith("volume_count_"):
        newname = col.split("_")[-1] + "_count"
        unstackedB.rename(columns={col:newname}, inplace=True)
    if col.startswith("volume_sum_"):
        newname = col.split("_")[-1] + "_volume"
        unstackedB.rename(columns={col:newname}, inplace=True)
unstackedB.rename(columns={"orgB":"object", "B_ID":"label"}, inplace=True)
unstackedB.set_index(['dataset', 'image_name', 'object', 'label'])

- combine and merge the data from **both** unstacked tables to include interaction metrics from all organelle objects

In [ ]:
contact_cnt = pd.concat([unstacked, unstackedB], axis=0).sort_index(axis=0)
contact_cnt = contact_cnt.groupby(['dataset', 'image_name', 'object', 'label']).sum().reset_index()
contact_cnt['label']=contact_cnt['label'].astype("Int64")

org_df = pd.merge(org_df, contact_cnt, how='left', on=['dataset', 'image_name', 'object', 'label'], sort=True)
org_df[contact_cnt.columns] = org_df[contact_cnt.columns].fillna(0)

In [10]:
contact_cnt_percell

volume_count   
dataset               image_name                                         orgA A_ID object                     
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER   1    ERXgolgi             104  \
                                                                                   ERXlyso               77   
                                                                                   ERXmito              874   
                                                                                   ERXperox             297   
                                                                         LD   209  LDXER                  1   
...                                                                                                     ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... mito 86   mitoXperox             1   
                                                                              87   mitoXperox             5   
                                                                              89   mitoXperox             1   
                                                                              93   mitoXperox             1   
                                                                              96   mitoXperox             2   

                                                                                               volume_sum  
dataset               image_name                                         orgA A_ID object                  
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER   1    ERXgolgi     58.950121  
                                                                                   ERXlyso      57.029118  
                                                                                   ERXmito     110.140125  
                                                                                   ERXperox     24.907430  
                                                                         LD   209  LDXER         0.002624  
...                                                                                                   ...  
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... mito 86   mitoXperox    0.015746  
                                                                              87   mitoXperox    0.136465  
                                                                              89   mitoXperox    0.002624  
                                                                              93   mitoXperox    0.018370  
                                                                              96   mitoXperox    0.076105  

[35587 rows x 2 columns]

In [9]:
unstacked

,dataset,image_name,object,label,ERXgolgi_count,ERXlyso_count,ERXmito_count,ERXperox_count,LDXER_count,LDXgolgi_count,...,LDXgolgi_volume,LDXlyso_volume,LDXmito_volume,LDXperox_volume,golgiXlyso_volume,golgiXmito_volume,golgiXperox_volume,lysoXmito_volume,lysoXperox_volume,mitoXperox_volume
0,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,ER,1,104.0,77.0,874.0,297.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,209,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,23,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,295,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,320,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,0.01837,0.023619,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27661,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,mito,86,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015746
27662,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,mito,87,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136465
27663,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,mito,89,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002624
27664,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,mito,93,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018370


In [8]:
contact_cnt

,dataset,image_name,object,label,ERXgolgi_count,ERXlyso_count,ERXmito_count,ERXperox_count,LDXER_count,LDXgolgi_count,...,LDXgolgi_volume,LDXlyso_volume,LDXmito_volume,LDXperox_volume,golgiXlyso_volume,golgiXmito_volume,golgiXperox_volume,lysoXmito_volume,lysoXperox_volume,mitoXperox_volume
0,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,ER,1,104.0,77.0,874.0,297.0,9.0,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,209,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,23,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,295,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,320,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.01837,0.023619,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102088,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,perox,95,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
102089,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,perox,96,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.002624
102090,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,perox,97,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
102091,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,perox,98,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.002624


In [11]:
org_df

,dataset,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,...,LDXgolgi_volume,LDXlyso_volume,LDXmito_volume,LDXperox_volume,golgiXlyso_volume,golgiXmito_volume,golgiXperox_volume,lysoXmito_volume,lysoXperox_volume,mitoXperox_volume
0,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,ER,1,"(0.4106, 0.0799, 0.0799)",6.109634,66.835593,63.892707,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,23,"(0.4106, 0.0799, 0.0799)",2.247069,9.690667,57.380142,5,113,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,46,"(0.4106, 0.0799, 0.0799)",1.826724,58.178574,24.628571,4,724,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,98,"(0.4106, 0.0799, 0.0799)",2.131930,34.215779,72.164467,5,424,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 11_cell 3_2...,LD,99,"(0.4106, 0.0799, 0.0799)",2.052970,34.756387,71.737585,5,430,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104834,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,perox,351,"(0.4106, 0.0799, 0.0799)",10.874520,67.504340,68.323191,26,842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005249,0.002624
104835,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,perox,352,"(0.4106, 0.0799, 0.0799)",11.219927,63.613480,69.818063,27,793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
104836,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,perox,353,"(0.4106, 0.0799, 0.0799)",11.275543,66.946388,64.956937,27,836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.005249
104837,20240118_C2-123_quant,20230817_C2-123_unconditioned_well 4_cell 1_un...,perox,354,"(0.4106, 0.0799, 0.0799)",11.788370,64.504606,67.654097,28,804,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.010497


## **`3` - Apply aggregate statistics for summarization**

- determine aggregate statistics to be applied per organelle object

In [12]:
###################
# summary stat group
###################

# ensure summary statistics are applied on a per organelle object level
group_by = ['dataset', 'image_name', 'object']

# metrics to be observed
sharedcolumns = ["SA_to_volume_ratio", "equivalent_diameter", "extent", "euler_number", "solidity", "axis_major_length"]

# statistical functions to be performed on the metrics
ag_func_standard = ['mean', 'median', 'std']

- summarize shared metrics between the organelle morphology and interaction tables

In [ ]:
###################
# summarize shared measurements between org_df and contacts_df
###################
org_cont_tabs = []
for tab in [org_df, contacts_df]:
    tab1 = tab[group_by + ['volume']].groupby(group_by).agg(['count', 'sum'] + ag_func_standard)
    tab2 = tab[group_by + ['surface_area']].groupby(group_by).agg(['sum'] + ag_func_standard)
    tab3 = tab[group_by + sharedcolumns].groupby(group_by).agg(ag_func_standard)
    shared_metrics = pd.merge(tab1, tab2, 'outer', on=group_by)
    shared_metrics = pd.merge(shared_metrics, tab3, 'outer', on=group_by)
    org_cont_tabs.append(shared_metrics)

org_summary = org_cont_tabs[0]
contact_summary = org_cont_tabs[1]

- summarize metrics in the region morphology table

In [ ]:
###################
# group metrics from regions_df similar to the above
###################
regions_summary = regions_df[group_by + ['volume', 'surface_area'] + sharedcolumns].set_index(group_by)

- summarize additional metrics in the organelle morphology table

In [ ]:
###################
# summarize extra metrics from org_df
###################
columns2 = [col for col in org_df.columns if col.endswith(("_count", "_volume"))]
contact_counts_summary = org_df[group_by + columns2].groupby(group_by).agg(['sum'] + ag_func_standard)
org_summary = pd.merge(org_summary, contact_counts_summary, 'outer', on=group_by)#left_on=group_by, right_on=True)

In [15]:
org_summary

volume   
                                                                                 count   
dataset               image_name                                         object          
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER          1  \
                                                                         LD         14   
                                                                         golgi      75   
                                                                         lyso       57   
                                                                         mito      457   
...                                                                                ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD         17   
                                                                         golgi      87   
                                                                         lyso      213   
                                                                         mito      353   
                                                                         perox     355   

                                                                                               
                                                                                         sum   
dataset               image_name                                         object                
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER      1259.650421  \
                                                                         LD         1.243928   
                                                                         golgi     79.734742   
                                                                         lyso      66.663000   
                                                                         mito     238.443174   
...                                                                                      ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD         0.558980   
                                                                         golgi     85.188081   
                                                                         lyso     199.996873   
                                                                         mito     491.501191   
                                                                         perox     81.991658   

                                                                                               
                                                                                        mean   
dataset               image_name                                         object                
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER      1259.650421  \
                                                                         LD         0.088852   
                                                                         golgi      1.063130   
                                                                         lyso       1.169526   
                                                                         mito       0.521757   
...                                                                                      ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD         0.032881   
                                                                         golgi      0.979173   
                                                                         lyso       0.938952   
                                                                         mito       1.392355   
                                                                         perox      0.230962   

                                                                                               
                                 

In [14]:
pd.set_option('display.max_columns', None)

## **`4` - Restructure distribution metrics tables**

- for XY-distribution collect summary statistics for voxel bins and wedges

- for Z-distribution collect summary statistics for voxel bins and wedges

> ###### Statistics collected: **mean, median, mode, minimum, maximum, range, standard deviation, skew, kurtosis and variance**

- calculate the coefficient of variation for the **mean**, **median**, **standard deviation** for the XY-distribution bin values

In [47]:
hist_dfs = []
for ind in range(0,1):#len(dist_df.index)):
    selection = dist_df.iloc[[ind]] #    selection = dist_df.loc[[ind]]
    bins_df = pd.DataFrame()
    wedges_df = pd.DataFrame()
    Z_df = pd.DataFrame()
    CV_df = pd.DataFrame()

    bins_df[['bins', 'masks', 'obj']] = selection[['XY_bins', 'XY_mask_vox_cnt_perbin', 'XY_obj_vox_cnt_perbin']]
    wedges_df[['bins', 'masks', 'obj']] = selection[['XY_wedges', 'XY_mask_vox_cnt_perwedge', 'XY_obj_vox_cnt_perwedge']]
    Z_df[['bins', 'masks', 'obj']] = selection[['Z_slices', 'Z_mask_vox_cnt', 'Z_obj_vox_cnt']]

    dfs = [selection[['dataset', 'image_name', 'object']].reset_index()]
    for df, prefix in zip([bins_df, wedges_df, Z_df], ["XY_bins_", "XY_wedges_", "Z_slices_"]):
        single_df = pd.DataFrame(list(zip(df["bins"].values[0][1:-1].split(", "), 
                                        df["obj"].values[0][1:-1].split(", "), 
                                        df["masks"].values[0][1:-1].split(", "))), columns =['bins', 'obj', 'mask']).astype(int)
        
        if "Z_" in prefix:
            single_df =  single_df.drop(single_df[single_df['mask'] == 0].index)
            single_df['bins'] = (single_df["bins"]/max(single_df.bins)*9.99).apply(np.floor)+1
            single_df = single_df.groupby("bins").agg(['sum']).reset_index()
            single_df.columns = ['bins',"obj","mask"]
    
        single_df['mask_fract'] = single_df['mask']/single_df['mask'].max()
        # single_df['obj_normed_tocell'] = (single_df["obj"]*single_df["mask_fract"]).fillna(0)
        single_df['obj_perc_per_bin'] = (single_df["obj"] / single_df["obj"].sum())*100
        single_df['obj_portion_normed_tobin'] = (single_df["obj_perc_per_bin"]/single_df["mask_fract"]).fillna(0)

        sumstats_df = pd.DataFrame()

        s = single_df['bins'].repeat(single_df['obj_portion_normed_tobin']*100)
        ###################################
        #SUB-STEPS 1 & 2
        ###################################
        sumstats_df['hist_mean']=[s.mean()]
        sumstats_df['hist_median']=[s.median()]
        if single_df['obj_portion_normed_tobin'].sum() != 0: sumstats_df['hist_mode']=[s.mode().iloc[0]]
        else: sumstats_df['hist_mode']=['NaN']
        sumstats_df['hist_min']=[s.min()]
        sumstats_df['hist_max']=[s.max()]
        sumstats_df['hist_range']=[s.max() - s.min()]
        sumstats_df['hist_stdev']=[s.std()]
        sumstats_df['hist_skew']=[s.skew()]
        sumstats_df['hist_kurtosis']=[s.kurtosis()]
        sumstats_df['hist_var']=[s.var()]
        sumstats_df.columns = [prefix+col for col in sumstats_df.columns]
        dfs.append(sumstats_df.reset_index())
    
    ###################################
    #SUB-STEP 3
    ###################################
    CV_df = pd.DataFrame(list(zip(selection["XY_obj_cv_perbin"].values[0][1:-1].split(", "))), columns =['CV']).astype(float)
    sumstats_CV_df = pd.DataFrame()
    sumstats_CV_df['XY_bin_CV_mean'] = CV_df.mean()
    sumstats_CV_df['XY_bin_CV_median'] = CV_df.median()
    sumstats_CV_df['XY_bin_CV_std'] = CV_df.std()
    dfs.append(sumstats_CV_df.reset_index().drop(['index'], axis=1))
    
    ###################################
    # Combine all resulting tables
    ###################################
    combined_df = pd.concat(dfs, axis=1).drop(columns="index")
    hist_dfs.append(combined_df)
dist_org_summary = pd.concat(hist_dfs, ignore_index=True)
dist_org_summary

,dataset,image_name,object,XY_bins_hist_mean,XY_bins_hist_median,XY_bins_hist_mode,XY_bins_hist_min,XY_bins_hist_max,XY_bins_hist_range,XY_bins_hist_stdev,XY_bins_hist_skew,XY_bins_hist_kurtosis,XY_bins_hist_var,XY_wedges_hist_mean,XY_wedges_hist_median,XY_wedges_hist_mode,XY_wedges_hist_min,XY_wedges_hist_max,XY_wedges_hist_range,XY_wedges_hist_stdev,XY_wedges_hist_skew,XY_wedges_hist_kurtosis,XY_wedges_hist_var,Z_slices_hist_mean,Z_slices_hist_median,Z_slices_hist_mode,Z_slices_hist_min,Z_slices_hist_max,Z_slices_hist_range,Z_slices_hist_stdev,Z_slices_hist_skew,Z_slices_hist_kurtosis,Z_slices_hist_var,XY_bin_CV_mean,XY_bin_CV_median,XY_bin_CV_std
0,20240220_C2-107_quant,20221027_C2-107_well 1_cell 1_untreated_Linear...,LD,4.286058,5.0,5,1,5,4,1.254602,-1.541087,0.958762,1.574027,3.142645,2.0,2,1,8,7,2.209255,0.929232,-0.685337,4.880806,2.389615,2.0,1.0,1.0,7.0,6.0,1.997943,1.546951,0.928892,3.991776,1.938065,1.835369,0.618022


- repeat the first two substeps for the nucleus distribution metrics

In [53]:
# nucleus distribution
nuc_dist_df = dist_df[["dataset", "image_name", 
                    "XY_bins", "XY_center_vox_cnt_perbin", "XY_mask_vox_cnt_perbin",
                    "XY_wedges", "XY_center_vox_cnt_perwedge", "XY_mask_vox_cnt_perwedge",
                    "Z_slices", "Z_center_vox_cnt", "Z_mask_vox_cnt"]].set_index(["dataset", "image_name"])
nuc_hist_dfs = []
for idx in nuc_dist_df.index.unique():
    selection = nuc_dist_df.loc[idx].iloc[[0]].reset_index()
    bins_df = pd.DataFrame()
    wedges_df = pd.DataFrame()
    Z_df = pd.DataFrame()

    bins_df[['bins', 'center', 'masks']] = selection[['XY_bins', 'XY_center_vox_cnt_perbin', 'XY_mask_vox_cnt_perbin']]
    wedges_df[['bins', 'center', 'masks']] = selection[['XY_wedges', 'XY_center_vox_cnt_perwedge', 'XY_mask_vox_cnt_perwedge']]
    Z_df[['bins', 'center', 'masks']] = selection[['Z_slices', 'Z_center_vox_cnt', 'Z_mask_vox_cnt']]

    dfs = [selection[['dataset', 'image_name']]]
    
    for df, prefix in zip([bins_df, wedges_df, Z_df], ["XY_bins_", "XY_wedges_", "Z_slices_"]):
        single_df = pd.DataFrame(list(zip(df["bins"].values[0][1:-1].split(", "), 
                                        df["masks"].values[0][1:-1].split(", "),
                                        df["center"].values[0][1:-1].split(", "))), columns =['bins', 'mask', 'obj']).astype(int)
        
        if "Z_" in prefix:
            single_df =  single_df.drop(single_df[single_df['mask'] == 0].index)
            single_df['bins'] = (single_df["bins"]/max(single_df.bins)*9.99).apply(np.floor)+1
            single_df = single_df.groupby("bins").agg(['sum']).reset_index()
            single_df.columns = ['bins',"mask","obj"]

        single_df['mask_fract'] = single_df['mask']/single_df['mask'].max()
        # single_df['obj_normed_tocell'] = (single_df["obj"]*single_df["mask_fract"]).fillna(0)
        single_df['obj_perc_per_bin'] = (single_df["obj"] / single_df["obj"].sum())*100
        single_df['obj_portion_normed_tobin'] = (single_df["obj_perc_per_bin"]/single_df["mask_fract"]).fillna(0)

        sumstats_df = pd.DataFrame()

        s = single_df['bins'].repeat(single_df['obj_portion_normed_tobin']*100)
        ###################################
        #SUB-STEPS 1 & 2 FOR NUC
        ###################################
        sumstats_df['hist_mean']=[s.mean()]
        sumstats_df['hist_median']=[s.median()]
        if single_df['obj_portion_normed_tobin'].sum() != 0: sumstats_df['hist_mode']=[s.mode().iloc[0]]
        else: sumstats_df['hist_mode']=['NaN']
        sumstats_df['hist_min']=[s.min()]
        sumstats_df['hist_max']=[s.max()]
        sumstats_df['hist_range']=[s.max() - s.min()]
        sumstats_df['hist_stdev']=[s.std()]
        sumstats_df['hist_skew']=[s.skew()]
        sumstats_df['hist_kurtosis']=[s.kurtosis()]
        sumstats_df['hist_var']=[s.var()]
        sumstats_df.columns = [prefix+col for col in sumstats_df.columns]
        dfs.append(sumstats_df.reset_index())
        
    ###################################
    # Combine all resulting tables
    ###################################
    combined_df = pd.concat(dfs, axis=1).drop(columns="index")
    nuc_hist_dfs.append(combined_df)
dist_center_summary = pd.concat(nuc_hist_dfs, ignore_index=True)
dist_center_summary.insert(2, column="object", value="nuc")
dist_center_summary

C:\Users\Shannon\AppData\Local\Temp\ipykernel_8540\4290006490.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  selection = nuc_dist_df.loc[idx].iloc[[0]].reset_index()
C:\Users\Shannon\AppData\Local\Temp\ipykernel_8540\4290006490.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  selection = nuc_dist_df.loc[idx].iloc[[0]].reset_index()
C:\Users\Shannon\AppData\Local\Temp\ipykernel_8540\4290006490.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  selection = nuc_dist_df.loc[idx].iloc[[0]].reset_index()
C:\Users\Shannon\AppData\Local\Temp\ipykernel_8540\4290006490.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  selection = nuc_dist_df.loc[idx].iloc[[0]].reset_index()
C:\Users\Shannon\AppData\Local\Temp\ipykernel_8540\4290006490.py:8: PerformanceWarning: indexing past lexsort depth may impact performance.
  selection = nuc_dist_df.loc[idx].iloc[[0]].reset_index()
C:\Us

,dataset,image_name,object,XY_bins_hist_mean,XY_bins_hist_median,XY_bins_hist_mode,XY_bins_hist_min,XY_bins_hist_max,XY_bins_hist_range,XY_bins_hist_stdev,...,Z_slices_hist_mean,Z_slices_hist_median,Z_slices_hist_mode,Z_slices_hist_min,Z_slices_hist_max,Z_slices_hist_range,Z_slices_hist_stdev,Z_slices_hist_skew,Z_slices_hist_kurtosis,Z_slices_hist_var
0,20240220_C2-107_quant,20221027_C2-107_well 1_cell 1_untreated_Linear...,nuc,1.0,1.0,1,1,1,0,0.0,...,7.294004,7.0,8.0,4.0,9.0,5.0,1.098648,-0.347350,-0.579546,1.207027
1,20240220_C2-107_quant,20221027_C2-107_well 1_cell 2_untreated_Linear...,nuc,1.0,1.0,1,1,1,0,0.0,...,6.261085,6.0,6.0,4.0,8.0,4.0,1.060288,0.113038,-1.002106,1.124210
2,20240220_C2-107_quant,20221027_C2-107_well 1_cell 3_untreated_Linear...,nuc,1.0,1.0,1,1,1,0,0.0,...,6.603834,7.0,7.0,4.0,9.0,5.0,1.090285,-0.356748,-0.527568,1.188722
3,20240220_C2-107_quant,20221027_C2-107_well 1_cell 4_untreated_Linear...,nuc,1.0,1.0,1,1,1,0,0.0,...,5.893096,6.0,6.0,3.0,8.0,5.0,1.293846,-0.100112,-0.813658,1.674037
4,20240220_C2-107_quant,20221027_C2-107_well 1_cell 5_untreated_Linear...,nuc,1.0,1.0,1,1,1,0,0.0,...,5.950500,6.0,6.0,3.0,8.0,5.0,1.201364,-0.097283,-0.765856,1.443274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 7_cell 1_50...,nuc,1.0,1.0,1,1,1,0,0.0,...,6.592731,7.0,7.0,4.0,9.0,5.0,1.042108,-0.248665,-0.533085,1.085988
137,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 7_cell 2_50...,nuc,1.0,1.0,1,1,1,0,0.0,...,6.169485,6.0,6.0,4.0,8.0,4.0,0.858970,-0.411233,-0.278896,0.737829
138,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 8_cell 3_un...,nuc,1.0,1.0,1,1,1,0,0.0,...,6.875607,7.0,7.0,4.0,9.0,5.0,1.085858,-0.179305,-0.516143,1.179087
139,20240118_C2-117_quant,20230622_C2-117_unconditioned_well 8_cell 4_un...,nuc,1.0,1.0,1,1,1,0,0.0,...,5.329227,5.0,1.0,1.0,10.0,9.0,2.970119,0.138550,-1.281238,8.821606


- combine nucleus and organelle distribution tables

In [ ]:
dist_summary = pd.concat([dist_org_summary, dist_center_summary], axis=0).set_index(group_by).sort_index()
dist_summary

## **`5` - Add normalized metrics**

- calculate fraction of cell area taken up by the organelles

In [ ]:
###################
# add normalization
###################
# organelle area fraction
area_fractions = []
for idx in org_summary.index.unique():
    org_vol = org_summary.loc[idx][('volume', 'sum')]
    cell_vol = regions_summary.loc[idx[:-1] + ('cell',)]["volume"]
    afrac = org_vol/cell_vol
    area_fractions.append(afrac)
org_summary[('volume', 'fraction')] = area_fractions
# TODO: add in line to reorder the level=0 columns here

- calculate fraction of organelle objects involved in specific interorganelle contacts

In [ ]:
# contact sites volume normalized
norm_toA_list = []
norm_toB_list = []
for col in contact_summary.index:
    norm_toA_list.append(contact_summary.loc[col][('volume', 'sum')]/org_summary.loc[col[:-1]+(col[-1].split('X')[0],)][('volume', 'sum')])
    norm_toB_list.append(contact_summary.loc[col][('volume', 'sum')]/org_summary.loc[col[:-1]+(col[-1].split('X')[1],)][('volume', 'sum')])
contact_summary[('volume', 'norm_to_A')] = norm_toA_list
contact_summary[('volume', 'norm_to_B')] = norm_toB_list

# number and area of individuals organelle involved in contact
cont_cnt = org_df[group_by]
cont_cnt[[col.split('_')[0] for col in org_df.columns if col.endswith(("_count"))]] = org_df[[col for col in org_df.columns if col.endswith(("_count"))]].astype(bool)
cont_cnt_perorg = cont_cnt.groupby(group_by).agg('sum')
cont_cnt_perorg.columns = pd.MultiIndex.from_product([cont_cnt_perorg.columns, ['count_in']])
for col in cont_cnt_perorg.columns:
    cont_cnt_perorg[(col[0], 'num_fraction_in')] = cont_cnt_perorg[col].values/org_summary[('volume', 'count')].values
cont_cnt_perorg.sort_index(axis=1, inplace=True)
org_summary = pd.merge(org_summary, cont_cnt_perorg, on=group_by, how='outer')

In [19]:
cont_cnt_perorg

ERXgolgi   
                                                                                count_in   
dataset               image_name                                         object            
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER            1  \
                                                                         LD            0   
                                                                         golgi        75   
                                                                         lyso          0   
                                                                         mito          0   
...                                                                                  ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD            0   
                                                                         golgi        76   
                                                                         lyso          0   
                                                                         mito          0   
                                                                         perox         0   

                                                                                                  
                                                                                num_fraction_in   
dataset               image_name                                         object                   
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER            1.000000  \
                                                                         LD            0.000000   
                                                                         golgi         1.000000   
                                                                         lyso          0.000000   
                                                                         mito          0.000000   
...                                                                                         ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD            0.000000   
                                                                         golgi         0.873563   
                                                                         lyso          0.000000   
                                                                         mito          0.000000   
                                                                         perox         0.000000   

                                                                                 ERXlyso   
                                                                                count_in   
dataset               image_name                                         object            
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER            1  \
                                                                         LD            0   
                                                                         golgi         0   
                                                                         lyso         53   
                                                                         mito          0   
...                                                                                  ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD            0   
                                                                         golgi         0   
                                                                         lyso        204   
                                                                         mito          0   
                                                                         perox         0   

                                                                                                  
                

In [17]:
org_summary

volume   
                                                                                 count   
dataset               image_name                                         object          
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER          1  \
                                                                         LD         14   
                                                                         golgi      75   
                                                                         lyso       57   
                                                                         mito      457   
...                                                                                ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD         17   
                                                                         golgi      87   
                                                                         lyso      213   
                                                                         mito      353   
                                                                         perox     355   

                                                                                               
                                                                                         sum   
dataset               image_name                                         object                
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER      1259.650421  \
                                                                         LD         1.243928   
                                                                         golgi     79.734742   
                                                                         lyso      66.663000   
                                                                         mito     238.443174   
...                                                                                      ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD         0.558980   
                                                                         golgi     85.188081   
                                                                         lyso     199.996873   
                                                                         mito     491.501191   
                                                                         perox     81.991658   

                                                                                               
                                                                                        mean   
dataset               image_name                                         object                
20240118_C2-117_quant 20230622_C2-117_unconditioned_well 11_cell 3_25... ER      1259.650421  \
                                                                         LD         0.088852   
                                                                         golgi      1.063130   
                                                                         lyso       1.169526   
                                                                         mito       0.521757   
...                                                                                      ...   
20240118_C2-123_quant 20230817_C2-123_unconditioned_well 4_cell 1_unt... LD         0.032881   
                                                                         golgi      0.979173   
                                                                         lyso       0.938952   
                                                                         mito       1.392355   
                                                                         perox      0.230962   

                                                                                               
                                 

## **`6` - Unstack and finalize summary stats tables**

- unstack and reorder organelle morphology summary table columns

In [15]:
###################
# flatten datasheets and combine
# TODO: restructure this so that all of the datasheets and unstacked and then reorded based on shared level 0 columns before flattening
###################
# org flattening
org_final = org_summary.unstack(-1)
for col in org_final.columns:
    if col[1] in ('count_in', 'num_fraction_in') or col[0].endswith(('_count', '_volume')):
        if col[2] not in col[0]:
            org_final.drop(col,axis=1, inplace=True)
new_col_order = ['dataset', 'image_name', 'object', 'volume', 'surface_area', 'SA_to_volume_ratio', 
                'equivalent_diameter', 'extent', 'euler_number', 'solidity', 'axis_major_length', 
                'ERXLD', 'ERXLD_count', 'ERXLD_volume', 'golgiXER', 'golgiXER_count', 'golgiXER_volume', 
                'golgiXLD', 'golgiXLD_count', 'golgiXLD_volume', 'golgiXperox', 'golgiXperox_count', 'golgiXperox_volume', 
                'lysoXER', 'lysoXER_count', 'lysoXER_volume', 'lysoXLD', 'lysoXLD_count', 'lysoXLD_volume', 
                'lysoXgolgi', 'lysoXgolgi_count', 'lysoXgolgi_volume', 'lysoXmito', 'lysoXmito_count', 'lysoXmito_volume', 
                'lysoXperox', 'lysoXperox_count', 'lysoXperox_volume', 'mitoXER', 'mitoXER_count', 'mitoXER_volume', 
                'mitoXLD', 'mitoXLD_count', 'mitoXLD_volume', 'mitoXgolgi', 'mitoXgolgi_count', 'mitoXgolgi_volume', 
                'mitoXperox', 'mitoXperox_count', 'mitoXperox_volume', 'peroxXER', 'peroxXER_count', 'peroxXER_volume', 
                'peroxXLD', 'peroxXLD_count', 'peroxXLD_volume']
new_cols = org_final.columns.reindex(new_col_order, level=0)
org_final = org_final.reindex(columns=new_cols[0])
org_final.columns = ["_".join((col_name[-1], col_name[1], col_name[0])) for col_name in org_final.columns.to_flat_index()]

- fill "NaN" values with 0 when necessary to final organelle morphology summary table

In [ ]:
#renaming, filling "NaN" with 0 when needed, and removing ER_std columns
for col in org_final.columns:
    if '_count_in_' or '_fraction_in_' in col:
        org_final[col] = org_final[col].fillna(0)
    if col.endswith(("_count_volume","_sum_volume", "_mean_volume", "_median_volume")):
        org_final[col] = org_final[col].fillna(0)
    if col.endswith("_count_volume"):
        org_final.rename(columns={col:col.split("_")[0]+"_count"}, inplace=True)
    if col.startswith("ER_std_"):
        org_final.drop(columns=[col], inplace=True)
org_final = org_final.reset_index()

- unstack and reorder organelle interactions summary table columns

In [ ]:
# contacts flattened
contact_final = contact_summary.unstack(-1)
contact_final.columns = ["_".join((col_name[-1], col_name[1], col_name[0])) for col_name in contact_final.columns.to_flat_index()]

- fill "NaN" values with 0 when necessary to final organelle interactions summary table

In [ ]:
#renaming and filling "NaN" with 0 when needed
for col in contact_final.columns:
    if col.endswith(("_count_volume","_sum_volume", "_mean_volume", "_median_volume")):
        contact_final[col] = contact_final[col].fillna(0)
    if col.endswith("_count_volume"):
        contact_final.rename(columns={col:col.split("_")[0]+"_count"}, inplace=True)
contact_final = contact_final.reset_index()

- unstack and reorder distribution measurements summary table columns to create finalized table

In [17]:
# distributions flattened
dist_final = dist_summary.unstack(-1)
dist_final.columns = ["_".join((col_name[1], col_name[0])) for col_name in dist_final.columns.to_flat_index()]
dist_final = dist_final.reset_index()

- unstack and reorder region morphology summary table columns

In [18]:
# regions flattened
regions_final = regions_summary.unstack(-1)
regions_final.columns = ["_".join((col_name[1], col_name[0])) for col_name in regions_final.columns.to_flat_index()]

ValueError: Index contains duplicate entries, cannot reshape

- add normalization to finalize region morphology summary table

In [ ]:
# normalization added
regions_final['nuc_area_fraction'] = regions_final['nuc_volume'] / regions_final['cell_volume']
regions_final = regions_final.reset_index()

- combine all four tables to create a complete summary table

In [ ]:
# combining them all
combined = pd.merge(org_final, contact_final, on=["dataset", "image_name"], how="outer")
combined = pd.merge(combined, dist_final, on=["dataset", "image_name"], how="outer")
combined = pd.merge(combined, regions_final, on=["dataset", "image_name"], how="outer").set_index(["dataset", "image_name"])
combined.columns = [col.replace('sum', 'total') for col in combined.columns]

## **`7` - Export summary stats tables as .csv files**

In [ ]:
###################
# export summary sheets
###################

# location for the final csv files to be exported to
out_path="D:/Paper 1 - Neuron vs astrocyte multispectral/Data/RAW summary data - UPDATED20240712"

# prefix added to summary tables
out_preffix="20240712_10per_neuron_"

org_summary.to_csv(out_path + f"/{out_preffix}per_org_summarystats.csv")
contact_summary.to_csv(out_path + f"/{out_preffix}per_contact_summarystats.csv")
dist_summary.to_csv(out_path + f"/{out_preffix}distribution_summarystats.csv")
regions_summary.to_csv(out_path + f"/{out_preffix}per_region_summarystats.csv")
combined.to_csv(out_path + f"/{out_preffix}summarystats_combined.csv")

# ***EXECUTE FUNCTION PROTOTYPE***

## **Define prototype `_batch_summary_stats` function**

In [55]:
def _batch_summary_stats(csv_path_list: List[str],
                         out_path: str,
                         out_preffix: str):
    """" 
    csv_path_list: List[str],
        A list of path strings where .csv files to analyze are located.
    out_path: str,
        A path string where the summary data file will be output to
    out_preffix: str
        The prefix used to name the output file.    
    """
    ds_count = 0
    fl_count = 0
    ###################
    # Read in the csv files and combine them into one of each type
    ###################
    org_tabs = []
    contact_tabs = []
    dist_tabs = []
    region_tabs = []

    for loc in csv_path_list:
        ds_count = ds_count + 1
        loc=Path(loc)
        files_store = sorted(loc.glob("*.csv"))
        for file in files_store:
            fl_count = fl_count + 1
            stem = file.stem

            org = "organelles"
            contacts = "contacts"
            dist = "distributions"
            regions = "_regions"

            if org in stem:
                test_orgs = pd.read_csv(file, index_col=0)
                test_orgs.insert(0, "dataset", stem[:-11])
                org_tabs.append(test_orgs)
            if contacts in stem:
                test_contact = pd.read_csv(file, index_col=0)
                test_contact.insert(0, "dataset", stem[:-9])
                contact_tabs.append(test_contact)
            if dist in stem:
                test_dist = pd.read_csv(file, index_col=0)
                test_dist.insert(0, "dataset", stem[:-14])
                dist_tabs.append(test_dist)
            if regions in stem:
                test_regions = pd.read_csv(file, index_col=0)
                test_regions.insert(0, "dataset", stem[:-8])
                region_tabs.append(test_regions)
            
    org_df = pd.concat(org_tabs,axis=0, join='outer')
    contacts_df = pd.concat(contact_tabs,axis=0, join='outer')
    dist_df = pd.concat(dist_tabs,axis=0, join='outer')
    regions_df = pd.concat(region_tabs,axis=0, join='outer')

    ###################
    # adding new metrics to the original sheets
    ###################
    # TODO: include these labels when creating the original sheets
    contact_cnt = contacts_df[["dataset", "image_name", "object", "label", "volume"]]
    contact_cnt[["orgA", "orgB"]] = contact_cnt["object"].str.split('X', expand=True)
    contact_cnt[["A_ID", "B_ID"]] = contact_cnt["label"].str.split('_', expand=True)
    contact_cnt["A"] = contact_cnt["orgA"] +"_" + contact_cnt["A_ID"].astype(str)
    contact_cnt["B"] = contact_cnt["orgB"] +"_" + contact_cnt["B_ID"].astype(str)

    contact_cnt_percell = contact_cnt[["dataset", "image_name", "orgA", "A_ID", "object", "volume"]].groupby(["dataset", "image_name", "orgA", "A_ID", "object"]).agg(["count", "sum"])
    contact_cnt_percell.columns = ["_".join(col_name).rstrip('_') for col_name in contact_cnt_percell.columns.to_flat_index()]
    unstacked = contact_cnt_percell.unstack(level='object')
    unstacked.columns = ["_".join(col_name).rstrip('_') for col_name in unstacked.columns.to_flat_index()]
    unstacked = unstacked.reset_index()
    for col in unstacked.columns:
        if col.startswith("volume_count_"):
            newname = col.split("_")[-1] + "_count"
            unstacked.rename(columns={col:newname}, inplace=True)
        if col.startswith("volume_sum_"):
            newname = col.split("_")[-1] + "_volume"
            unstacked.rename(columns={col:newname}, inplace=True)
    unstacked.rename(columns={"orgA":"object", "A_ID":"label"}, inplace=True)
    unstacked.set_index(['dataset', 'image_name', 'object', 'label'])

    contact_percellB = contact_cnt[["dataset", "image_name", "orgB", "B_ID", "object", "volume"]].groupby(["dataset", "image_name", "orgB", "B_ID", "object"]).agg(["count", "sum"])
    contact_percellB.columns = ["_".join(col_name).rstrip('_') for col_name in contact_percellB.columns.to_flat_index()]
    unstackedB = contact_percellB.unstack(level='object')
    unstackedB.columns = ["_".join(col_name).rstrip('_') for col_name in unstackedB.columns.to_flat_index()]
    unstackedB = unstackedB.reset_index()
    for col in unstackedB.columns:
        if col.startswith("volume_count_"):
            newname = col.split("_")[-1] + "_count"
            unstackedB.rename(columns={col:newname}, inplace=True)
        if col.startswith("volume_sum_"):
            newname = col.split("_")[-1] + "_volume"
            unstackedB.rename(columns={col:newname}, inplace=True)
    unstackedB.rename(columns={"orgB":"object", "B_ID":"label"}, inplace=True)
    unstackedB.set_index(['dataset', 'image_name', 'object', 'label'])

    contact_cnt = pd.concat([unstacked, unstackedB], axis=0).sort_index(axis=0)
    contact_cnt = contact_cnt.groupby(['dataset', 'image_name', 'object', 'label']).sum().reset_index()
    contact_cnt['label']=contact_cnt['label'].astype("Int64")

    org_df = pd.merge(org_df, contact_cnt, how='left', on=['dataset', 'image_name', 'object', 'label'], sort=True)
    org_df[contact_cnt.columns] = org_df[contact_cnt.columns].fillna(0)

    ###################
    # summary stat group
    ###################
    group_by = ['dataset', 'image_name', 'object']
    sharedcolumns = ["SA_to_volume_ratio", "equivalent_diameter", "extent", "euler_number", "solidity", "axis_major_length"]
    ag_func_standard = ['mean', 'median', 'std']

    ###################
    # summarize shared measurements between org_df and contacts_df
    ###################
    org_cont_tabs = []
    for tab in [org_df, contacts_df]:
        tab1 = tab[group_by + ['volume']].groupby(group_by).agg(['count', 'sum'] + ag_func_standard)
        tab2 = tab[group_by + ['surface_area']].groupby(group_by).agg(['sum'] + ag_func_standard)
        tab3 = tab[group_by + sharedcolumns].groupby(group_by).agg(ag_func_standard)
        shared_metrics = pd.merge(tab1, tab2, 'outer', on=group_by)
        shared_metrics = pd.merge(shared_metrics, tab3, 'outer', on=group_by)
        org_cont_tabs.append(shared_metrics)

    org_summary = org_cont_tabs[0]
    contact_summary = org_cont_tabs[1]

    ###################
    # group metrics from regions_df similar to the above
    ###################
    regions_summary = regions_df[group_by + ['volume', 'surface_area'] + sharedcolumns].set_index(group_by)

    ###################
    # summarize extra metrics from org_df
    ###################
    columns2 = [col for col in org_df.columns if col.endswith(("_count", "_volume"))]
    contact_counts_summary = org_df[group_by + columns2].groupby(group_by).agg(['sum'] + ag_func_standard)
    org_summary = pd.merge(org_summary, contact_counts_summary, 'outer', on=group_by)#left_on=group_by, right_on=True)

    ###################
    # summarize distribution measurements
    ###################
    # organelle distributions
    hist_dfs = []
    for ind in range(0,len(dist_df.index)):
        selection = dist_df.iloc[[ind]] #    selection = dist_df.loc[[ind]]
        bins_df = pd.DataFrame()
        wedges_df = pd.DataFrame()
        Z_df = pd.DataFrame()
        CV_df = pd.DataFrame()

        bins_df[['bins', 'masks', 'obj']] = selection[['XY_bins', 'XY_mask_vox_cnt_perbin', 'XY_obj_vox_cnt_perbin']]
        wedges_df[['bins', 'masks', 'obj']] = selection[['XY_wedges', 'XY_mask_vox_cnt_perwedge', 'XY_obj_vox_cnt_perwedge']]
        Z_df[['bins', 'masks', 'obj']] = selection[['Z_slices', 'Z_mask_vox_cnt', 'Z_obj_vox_cnt']]

        dfs = [selection[['dataset', 'image_name', 'object']].reset_index()]
        for df, prefix in zip([bins_df, wedges_df, Z_df], ["XY_bins_", "XY_wedges_", "Z_slices_"]):
            single_df = pd.DataFrame(list(zip(df["bins"].values[0][1:-1].split(", "), 
                                            df["obj"].values[0][1:-1].split(", "), 
                                            df["masks"].values[0][1:-1].split(", "))), columns =['bins', 'obj', 'mask']).astype(int)
            
            if "Z_" in prefix:
                single_df =  single_df.drop(single_df[single_df['mask'] == 0].index)
                single_df['bins'] = (single_df["bins"]/max(single_df.bins)*9.99).apply(np.floor)+1
                single_df = single_df.groupby("bins").agg(['sum']).reset_index()
                single_df.columns = ['bins',"obj","mask"]
        
            single_df['mask_fract'] = single_df['mask']/single_df['mask'].max()
            # single_df['obj_normed_tocell'] = (single_df["obj"]*single_df["mask_fract"]).fillna(0)
            single_df['obj_perc_per_bin'] = (single_df["obj"] / single_df["obj"].sum())*100
            single_df['obj_portion_normed_tobin'] = (single_df["obj_perc_per_bin"]/single_df["mask_fract"]).fillna(0)

            sumstats_df = pd.DataFrame()

            s = single_df['bins'].repeat(single_df['obj_portion_normed_tobin']*100)

            sumstats_df['hist_mean']=[s.mean()]
            sumstats_df['hist_median']=[s.median()]
            if single_df['obj_portion_normed_tobin'].sum() != 0: sumstats_df['hist_mode']=[s.mode().iloc[0]]
            else: sumstats_df['hist_mode']=['NaN']
            sumstats_df['hist_min']=[s.min()]
            sumstats_df['hist_max']=[s.max()]
            sumstats_df['hist_range']=[s.max() - s.min()]
            sumstats_df['hist_stdev']=[s.std()]
            sumstats_df['hist_skew']=[s.skew()]
            sumstats_df['hist_kurtosis']=[s.kurtosis()]
            sumstats_df['hist_var']=[s.var()]
            sumstats_df.columns = [prefix+col for col in sumstats_df.columns]
            dfs.append(sumstats_df.reset_index())

        CV_df = pd.DataFrame(list(zip(selection["XY_obj_cv_perbin"].values[0][1:-1].split(", "))), columns =['CV']).astype(float)
        sumstats_CV_df = pd.DataFrame()
        sumstats_CV_df['XY_bin_CV_mean'] = CV_df.mean()
        sumstats_CV_df['XY_bin_CV_median'] = CV_df.median()
        sumstats_CV_df['XY_bin_CV_std'] = CV_df.std()
        dfs.append(sumstats_CV_df.reset_index().drop(['index'], axis=1))

        combined_df = pd.concat(dfs, axis=1).drop(columns="index")
        hist_dfs.append(combined_df)
    dist_org_summary = pd.concat(hist_dfs, ignore_index=True)
    dist_org_summary

    # nucleus distribution
    nuc_dist_df = dist_df[["dataset", "image_name", 
                        "XY_bins", "XY_center_vox_cnt_perbin", "XY_mask_vox_cnt_perbin",
                        "XY_wedges", "XY_center_vox_cnt_perwedge", "XY_mask_vox_cnt_perwedge",
                        "Z_slices", "Z_center_vox_cnt", "Z_mask_vox_cnt"]].set_index(["dataset", "image_name"])
    nuc_hist_dfs = []
    for idx in nuc_dist_df.index.unique():
        selection = nuc_dist_df.loc[idx].iloc[[0]].reset_index()
        bins_df = pd.DataFrame()
        wedges_df = pd.DataFrame()
        Z_df = pd.DataFrame()

        bins_df[['bins', 'center', 'masks']] = selection[['XY_bins', 'XY_center_vox_cnt_perbin', 'XY_mask_vox_cnt_perbin']]
        wedges_df[['bins', 'center', 'masks']] = selection[['XY_wedges', 'XY_center_vox_cnt_perwedge', 'XY_mask_vox_cnt_perwedge']]
        Z_df[['bins', 'center', 'masks']] = selection[['Z_slices', 'Z_center_vox_cnt', 'Z_mask_vox_cnt']]

        dfs = [selection[['dataset', 'image_name']]]
        for df, prefix in zip([bins_df, wedges_df, Z_df], ["XY_bins_", "XY_wedges_", "Z_slices_"]):
            single_df = pd.DataFrame(list(zip(df["bins"].values[0][1:-1].split(", "), 
                                            df["masks"].values[0][1:-1].split(", "),
                                            df["center"].values[0][1:-1].split(", "))), columns =['bins', 'mask', 'obj']).astype(int)

            if "Z_" in prefix:
                single_df =  single_df.drop(single_df[single_df['mask'] == 0].index)
                single_df['bins'] = (single_df["bins"]/max(single_df.bins)*9.99).apply(np.floor)+1
                single_df = single_df.groupby("bins").agg(['sum']).reset_index()
                single_df.columns = ['bins',"mask","obj"]
        
            single_df['mask_fract'] = single_df['mask']/single_df['mask'].max()
            # single_df['obj_normed_tocell'] = (single_df["obj"]*single_df["mask_fract"]).fillna(0)
            single_df['obj_perc_per_bin'] = (single_df["obj"] / single_df["obj"].sum())*100
            single_df['obj_portion_normed_tobin'] = (single_df["obj_perc_per_bin"]/single_df["mask_fract"]).fillna(0)

            sumstats_df = pd.DataFrame()

            s = single_df['bins'].repeat(single_df['obj_portion_normed_tobin']*100)

            sumstats_df['hist_mean']=[s.mean()]
            sumstats_df['hist_median']=[s.median()]
            if single_df['obj_portion_normed_tobin'].sum() != 0: sumstats_df['hist_mode']=[s.mode().iloc[0]]
            else: sumstats_df['hist_mode']=['NaN']
            sumstats_df['hist_min']=[s.min()]
            sumstats_df['hist_max']=[s.max()]
            sumstats_df['hist_range']=[s.max() - s.min()]
            sumstats_df['hist_stdev']=[s.std()]
            sumstats_df['hist_skew']=[s.skew()]
            sumstats_df['hist_kurtosis']=[s.kurtosis()]
            sumstats_df['hist_var']=[s.var()]
            sumstats_df.columns = [prefix+col for col in sumstats_df.columns]
            dfs.append(sumstats_df.reset_index())
        combined_df = pd.concat(dfs, axis=1).drop(columns="index")
        nuc_hist_dfs.append(combined_df)
    dist_center_summary = pd.concat(nuc_hist_dfs, ignore_index=True)
    dist_center_summary.insert(2, column="object", value="nuc")

    dist_summary = pd.concat([dist_org_summary, dist_center_summary], axis=0).set_index(group_by).sort_index()


    ###################
    # add normalization
    ###################
    # organelle area fraction
    area_fractions = []
    for idx in org_summary.index.unique():
        org_vol = org_summary.loc[idx][('volume', 'sum')]
        cell_vol = regions_summary.loc[idx[:-1] + ('cell',)]["volume"]
        afrac = org_vol/cell_vol
        area_fractions.append(afrac)
    org_summary[('volume', 'fraction')] = area_fractions
    # TODO: add in line to reorder the level=0 columns here

    # contact sites volume normalized
    norm_toA_list = []
    norm_toB_list = []
    for col in contact_summary.index:
        norm_toA_list.append(contact_summary.loc[col][('volume', 'sum')]/org_summary.loc[col[:-1]+(col[-1].split('X')[0],)][('volume', 'sum')])
        norm_toB_list.append(contact_summary.loc[col][('volume', 'sum')]/org_summary.loc[col[:-1]+(col[-1].split('X')[1],)][('volume', 'sum')])
    contact_summary[('volume', 'norm_to_A')] = norm_toA_list
    contact_summary[('volume', 'norm_to_B')] = norm_toB_list

    # number and area of individuals organelle involved in contact
    cont_cnt = org_df[group_by]
    cont_cnt[[col.split('_')[0] for col in org_df.columns if col.endswith(("_count"))]] = org_df[[col for col in org_df.columns if col.endswith(("_count"))]].astype(bool)
    cont_cnt_perorg = cont_cnt.groupby(group_by).agg('sum')
    cont_cnt_perorg.columns = pd.MultiIndex.from_product([cont_cnt_perorg.columns, ['count_in']])
    for col in cont_cnt_perorg.columns:
        cont_cnt_perorg[(col[0], 'num_fraction_in')] = cont_cnt_perorg[col].values/org_summary[('volume', 'count')].values
    cont_cnt_perorg.sort_index(axis=1, inplace=True)
    org_summary = pd.merge(org_summary, cont_cnt_perorg, on=group_by, how='outer')


    ###################
    # flatten datasheets and combine
    # TODO: restructure this so that all of the datasheets and unstacked and then reorded based on shared level 0 columns before flattening
    ###################
    # org flattening
    org_final = org_summary.unstack(-1)
    for col in org_final.columns:
        if col[1] in ('count_in', 'num_fraction_in') or col[0].endswith(('_count', '_volume')):
            if col[2] not in col[0]:
                org_final.drop(col,axis=1, inplace=True)
    new_col_order = ['dataset', 'image_name', 'object', 'volume', 'surface_area', 'SA_to_volume_ratio', 
                 'equivalent_diameter', 'extent', 'euler_number', 'solidity', 'axis_major_length', 
                 'ERXLD', 'ERXLD_count', 'ERXLD_volume', 'golgiXER', 'golgiXER_count', 'golgiXER_volume', 
                 'golgiXLD', 'golgiXLD_count', 'golgiXLD_volume', 'golgiXperox', 'golgiXperox_count', 'golgiXperox_volume', 
                 'lysoXER', 'lysoXER_count', 'lysoXER_volume', 'lysoXLD', 'lysoXLD_count', 'lysoXLD_volume', 
                 'lysoXgolgi', 'lysoXgolgi_count', 'lysoXgolgi_volume', 'lysoXmito', 'lysoXmito_count', 'lysoXmito_volume', 
                 'lysoXperox', 'lysoXperox_count', 'lysoXperox_volume', 'mitoXER', 'mitoXER_count', 'mitoXER_volume', 
                 'mitoXLD', 'mitoXLD_count', 'mitoXLD_volume', 'mitoXgolgi', 'mitoXgolgi_count', 'mitoXgolgi_volume', 
                 'mitoXperox', 'mitoXperox_count', 'mitoXperox_volume', 'peroxXER', 'peroxXER_count', 'peroxXER_volume', 
                 'peroxXLD', 'peroxXLD_count', 'peroxXLD_volume']
    new_cols = org_final.columns.reindex(new_col_order, level=0)
    org_final = org_final.reindex(columns=new_cols[0])
    org_final.columns = ["_".join((col_name[-1], col_name[1], col_name[0])) for col_name in org_final.columns.to_flat_index()]

    #renaming, filling "NaN" with 0 when needed, and removing ER_std columns
    for col in org_final.columns:
        if '_count_in_' or '_fraction_in_' in col:
            org_final[col] = org_final[col].fillna(0)
        if col.endswith(("_count_volume","_sum_volume", "_mean_volume", "_median_volume")):
            org_final[col] = org_final[col].fillna(0)
        if col.endswith("_count_volume"):
            org_final.rename(columns={col:col.split("_")[0]+"_count"}, inplace=True)
        if col.startswith("ER_std_"):
            org_final.drop(columns=[col], inplace=True)
    org_final = org_final.reset_index()

    # contacts flattened
    contact_final = contact_summary.unstack(-1)
    contact_final.columns = ["_".join((col_name[-1], col_name[1], col_name[0])) for col_name in contact_final.columns.to_flat_index()]

    #renaming and filling "NaN" with 0 when needed
    for col in contact_final.columns:
        if col.endswith(("_count_volume","_sum_volume", "_mean_volume", "_median_volume")):
            contact_final[col] = contact_final[col].fillna(0)
        if col.endswith("_count_volume"):
            contact_final.rename(columns={col:col.split("_")[0]+"_count"}, inplace=True)
    contact_final = contact_final.reset_index()

    # distributions flattened
    dist_final = dist_summary.unstack(-1)
    dist_final.columns = ["_".join((col_name[1], col_name[0])) for col_name in dist_final.columns.to_flat_index()]
    dist_final = dist_final.reset_index()

    # regions flattened & normalization added
    regions_final = regions_summary.unstack(-1)
    regions_final.columns = ["_".join((col_name[1], col_name[0])) for col_name in regions_final.columns.to_flat_index()]
    regions_final['nuc_area_fraction'] = regions_final['nuc_volume'] / regions_final['cell_volume']
    regions_final = regions_final.reset_index()

    # combining them all
    combined = pd.merge(org_final, contact_final, on=["dataset", "image_name"], how="outer")
    combined = pd.merge(combined, dist_final, on=["dataset", "image_name"], how="outer")
    combined = pd.merge(combined, regions_final, on=["dataset", "image_name"], how="outer").set_index(["dataset", "image_name"])
    combined.columns = [col.replace('sum', 'total') for col in combined.columns]

    ###################
    # export summary sheets
    ###################
    org_summary.to_csv(out_path + f"/{out_preffix}per_org_summarystats.csv")
    contact_summary.to_csv(out_path + f"/{out_preffix}per_contact_summarystats.csv")
    dist_summary.to_csv(out_path + f"/{out_preffix}distribution_summarystats.csv")
    regions_summary.to_csv(out_path + f"/{out_preffix}per_region_summarystats.csv")
    combined.to_csv(out_path + f"/{out_preffix}summarystats_combined.csv")

    print(f"Processing of {fl_count} files from {ds_count} dataset(s) is complete.")
    return f"{fl_count} files from {ds_count} dataset(s) were processed"

## **Run prototype `_batch_summary_stats` function**

In [57]:
out=_batch_summary_stats(csv_path_list=["E:/Experiments (C2-109 - C2-116)/Paper1-data_CHH_VD\ImagingData/VD_data/VD-0428/20230918_VD-0428_3D-analysis/20240328_VD-0428_quant",
                                        "E:/Experiments (C2-109 - C2-116)/Paper1-data_CHH_VD\ImagingData/VD_data/VD-0505/20230920_VD-0505_3D-analysis/20240401_VD-0505_quant",
                                        "E:/Experiments (C2-109 - C2-116)/Paper1-data_CHH_VD\ImagingData/VD_data/VD-0512/20230920_VD-0512_3D-analysis/20240404_VD-0512_quant",
                                        "E:/Experiments (C2-109 - C2-116)/Paper1-data_CHH_VD\ImagingData/VD_data/VD-0609/20230921_VD-0609_3D-analysis/20240415_VD-0609_quant"],
                         out_path="D:/Paper 1 - Neuron vs astrocyte multispectral/Data/RAW summary data - UPDATED20240712",
                         out_preffix="20240712_10per_neuron_")

C:\Users\Shannon\AppData\Local\Temp\ipykernel_8540\204962698.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contact_cnt[["orgA", "orgB"]] = contact_cnt["object"].str.split('X', expand=True)
C:\Users\Shannon\AppData\Local\Temp\ipykernel_8540\204962698.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contact_cnt[["orgA", "orgB"]] = contact_cnt["object"].str.split('X', expand=True)
C:\Users\Shannon\AppData\Local\Temp\ipykernel_8540\204962698.py:63: SettingWithCopyWarning: 
A value is trying to be set

Processing of 20 files from 5 dataset(s) is complete.
